In [1]:
from ecell4 import *
from ecell4.extra import ensemble
import numpy as np
# import matplotlib.pylab as plt

import logging
_logger = logging.getLogger(__name__)

def singlerun(L, Nas, Nbs, rm, D, k0, duration, solver='ode',simrun=1,nproc=1,method='serial'):
    _logger.info(solver)
    V = L ** 3
    M = max(int(min(cbrt(Nas + Nbs), L / (2 * rm))), 3)
    if solver == 'spa':
        solver=("spatiocyte", rm)
        V = spatiocyte.SpatiocyteWorld.calculate_volume(Real3(L, L, L), rm)
    elif solver == 'egfrd':
        solver = ('egfrd', Integer3(M, M, M))

    kon = k0
    keff = k0 / (Nbs / V) / (Nas / V)
    kd = 4 * np.pi * (rm * 2) * (D * 2)
    ka = keff * kd / (kd - keff)
    assert(ka > 0)

    with species_attributes():
        A | {'D': D, 'radius': rm}
        B | {'D': D, 'radius': rm}

    with reaction_rules():
        ~A > A | kon
        #A + B > B | ka
        A > ~A | keff
    m = get_model()

    _logger.info('av={:.3},Nas={},Nbs={}'.format(float(V), Nas, Nbs))
    _logger.info('kon={:.3},keff={:.3},kd={:.3},ka={:.3},kfactor={:.3}'.format(kon, keff, kd, ka, ka / kd))
    res = ensemble.ensemble_simulations(duration, n=simrun, y0={'B': Nbs}, model=m, solver=solver, volume=ones()*L, species_list=["A"], return_type='observer', nproc=nproc, method=method, path='tmp')
    res.save('res_{:.3f}_{:.3f}.csv'.format(rm, D))
    _logger.info(res)


if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)

    simrun = 10#0
    duration = 1000
    Nas = 5#00
    Nbs = 1#00
    nproc = min(simrun, 300)
    L = 1#cbrt(100.0)
    k0 = 0.1
    method = 'multiprocessing'

    singlerun(L, Nas, Nbs, 0.05, 0.1, k0, duration, 'egfrd', simrun, nproc, method)
    #singlerun(L, Nas, Nbs, 0.05, 0.01, k0, duration, 'egfrd', simrun, nproc, method)
    #singlerun(L, Nas, Nbs, 0.005, 0.1, k0, duration, 'egfrd', simrun, nproc, method)

INFO:__main__:egfrd
INFO:__main__:av=1.0,Nas=5,Nbs=1
INFO:__main__:kon=0.1,keff=0.02,kd=0.251,ka=0.0217,kfactor=0.0865
INFO:__main__:<ecell4.extra.ensemble.ensemble_simulations.<locals>.DummyObserver object at 0x7efc13f667b8>


In [6]:
1-0.3*0.74/0.515

0.5689320388349515